Apriori Algorithm from Scratch 

Apriori gets the association rules which are based on minimum support and minimum confidence. Minimum support is the occurrence of an item in the transaction to the total number of transactions, this makes the rules. If a rule is A --> B than the confidence is, occurrence of B to the occurrence of A union B.


In [ ]:
import csv
from itertools import combinations

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load data and store the transactions in dictionary

In [ ]:
def load_data(trasanction_file='/content/gdrive/MyDrive/datamining/grocery.csv'):
    transactions = dict()
    with open(trasanction_file) as f:
        transaction_data = csv.reader(f, delimiter=',')
        count = 0
        for line in transaction_data:
            count += 1
            transactions[count] = list(set(line))
    return transactions

In [ ]:
load_data()

{1: ['BISCUIT', 'MILK', 'BREAD'],
 2: ['BISCUIT', 'MILK', 'BREAD', 'CORNFLAKES'],
 3: ['TEA', 'BOURNVITA', 'BREAD'],
 4: ['MAGGI', 'MILK', 'BREAD', 'JAM'],
 5: ['TEA', 'MAGGI', 'BISCUIT'],
 6: ['TEA', 'BOURNVITA', 'BREAD'],
 7: ['TEA', 'MAGGI', 'CORNFLAKES'],
 8: ['TEA', 'MAGGI', 'BISCUIT', 'BREAD'],
 9: ['TEA', 'MAGGI', 'BREAD', 'JAM'],
 10: ['MILK', 'BREAD'],
 11: ['BISCUIT', 'COCK', 'CORNFLAKES', 'COFFEE'],
 12: ['BISCUIT', 'COCK', 'CORNFLAKES', 'COFFEE'],
 13: ['BOURNVITA', 'SUGER', 'COFFEE'],
 14: ['COCK', 'BREAD', 'COFFEE'],
 15: ['BISCUIT', 'BREAD', 'SUGER'],
 16: ['CORNFLAKES', 'SUGER', 'COFFEE'],
 17: ['BOURNVITA', 'BREAD', 'SUGER'],
 18: ['BREAD', 'SUGER', 'COFFEE'],
 19: ['BREAD', 'SUGER', 'COFFEE'],
 20: ['TEA', 'CORNFLAKES', 'MILK', 'COFFEE']}

get the frequency of items 

In [ ]:
def item_frequency(items_list, transactions, check=False):
    items_freq = dict()
    for i in items_list:
        temp = {i}
        if check:
            temp = set(i)
        for j in transactions.items():
            if temp.issubset(set(j[1])):
                if i in items_freq:
                    items_freq[i] += 1
                else:
                    items_freq[i] = 1
    return items_freq


calculate support for the items

In [ ]:
def support(items_counts, trans):
    support = dict()
    total_trans = len(trans)
    for i in items_counts:
        support[i] = items_counts[i]/total_trans
    return support
    

Get the associlation rules

In [ ]:
def getAssociation_rules(items):
    rules = []
    assoc_rules = {}
    for i in items:
        assoc_rules = {}
        if type(i) != type(str()):
            i = list(i)
            temp = i[:]
            for j in range(len(i)):
                k = temp[j]
                del temp[j]
                assoc_rules[k] = temp
                temp = i[:]
        rules.append(assoc_rules)
    rules_new = []
    for i in rules:
        for j in i.items():
            if type(j[1]) != type(str()):
                rules_new.append({tuple(j[1])[0]: j[0]})
            else:
                rules_new.append({j[1]: j[0]})
    rules.extend(rules_new)
    return rules


calculate the confidence for all the new rules 

In [ ]:
def confidence(associations, d, min_confidence):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])

            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                        down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                        elif len(right) <= len({k}):
                            if {k} - right == set():    
                                down = d[k]
            if up/down >= min_confidence:
                ans[tuple(left)[0]] = right, up/down, up, down
    return ans

get all the rules with minimum support 0.0.5 and minimum confidence 0.7

In [ ]:
min_support = 0.05
min_confidence = 0.7
file_loc = "/content/gdrive/MyDrive/datamining/grocery.csv"
trans = load_data()
number_of_trans = [len(i) for i in trans.values()]
items_lst = set()

itemcount_track = list()
    
for i in trans.values():
  for j in i:
    items_lst.add(j)

store_item_lst = list(items_lst)[:]
items_with_min_support = list()
items_counts = item_frequency(items_lst, trans)
itemcount_track.append(items_counts)
items_with_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})

for i in range(2, max(number_of_trans)+1):
  item_list = combinations(items_lst, i)
  items_counts = item_frequency(item_list, trans, check=True)
  itemcount_track.append(items_counts)
  if list({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support}.keys()) != []:
    items_with_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
d = {}
{d.update(i) for i in itemcount_track}
associations = getAssociation_rules(items_with_min_support[len(items_with_min_support)-1])
associations_with_min_confidene = confidence(associations, d, min_confidence)

print(associations_with_min_confidene)


{'BISCUIT': ({'CORNFLAKES', 'COCK', 'COFFEE'}, 1.0, 2, 2), 'COCK': ({'BISCUIT', 'CORNFLAKES', 'COFFEE'}, 1.0, 2, 2), 'CORNFLAKES': ({'BISCUIT', 'COCK', 'COFFEE'}, 1.0, 2, 2), 'COFFEE': ({'BISCUIT', 'COCK', 'CORNFLAKES'}, 1.0, 2, 2)}
